In [213]:
import numpy as np
from itertools import product
from scipy.stats import norm
from scipy.special import logsumexp
from numpy import linalg as LA


In [168]:
def data_generation(T,p0,mu,sig,n_step):
    
    xs = np.zeros(n_step)
    zs = np.zeros(n_step,dtype='int')
    for i in range(n_step):
        if i==0:zs[i] = np.random.choice(np.arange(len(p0)),replace=False,p = p0)
        else: zs[i] = np.random.choice(np.arange(len(p0)),replace=False,p = T[zs[i-1]])
        xs[i] = np.random.randn()*sig+mu[zs[i]]
    return zs,xs

In [203]:
# joint_loglik = log p(z_0)\sum logp(z_i|z_i-1) \sum logp(x_t|z_t)
def joint_loglik(T,p0,mu,sig,x,z): 
    return np.sum(np.log(p0[z]))+np.sum(norm.logpdf(x,loc = mu[z],scale = sig))
# transition kernel of MH
# Suppose q(z|z') =q(z'|z) = 1/n (uniformly random)
def transition_kernel(T,p0,mu,sig,x):
    n_step = len(x)
    k = len(p0)
    # transition kernel size k^t * k^t
    n = k**n_step
    print ('transition kernel size {}*{}'.format(k**n_step,k**n_step))
    states = np.vstack(list(product(range(k),repeat = n_step)))
    assert len(states)==n
    t_mh = np.ones([n,n])*-np.inf
    for i in range(n):
        for j in range(i+1,n):
            # calculate logp(z,x)+logq(z|z')-logp(z',x)-logq(z'|z)
            diff = joint_loglik(T,p0,mu,sig,x,states[i])-joint_loglik(T,p0,mu,sig,x,states[j])
            t_mh[i][j] = np.log(1./n)+np.min([0,diff])
            t_mh[j][i] = np.log(1./n)+np.min([0,-diff])
    for i in range(n):
        t_mh[i][i] = np.log(1-np.exp(logsumexp(t_mh[:,i])))
    return t_mh

In [220]:
# transition T for latent variable z0
T = np.array([[0.1,0.9],[0.9,0.1]])
# initial distribution
p0 = np.array([0.3,0.7])
# p(x_t|z_t)
mu = np.array([-1.,1.])
sig = np.sqrt(0.5)

n_step = 2

for _ in range(10):

    # time length
    zs,xs = data_generation(T,p0,mu,sig,n_step)

    n_step = 2
    t_mh=transition_kernel(T,p0,mu,sig,xs)
    #### Column sum to 1
    assert np.sum(np.exp(t_mh),axis=0).all()==1.

    w, v = LA.eig(np.exp(t_mh))
    w = np.sort(w)[::-1]
    print ('eigen_vs',w,'omega',w[1])

transition kernel size 4*4
eigen_vs [1.         0.74837243 0.47670517 0.24851116] omega 0.7483724256174284
transition kernel size 4*4
eigen_vs [1.         0.7287777  0.37134896 0.23598822] omega 0.7287777004935975
transition kernel size 4*4
eigen_vs [1.         0.61533297 0.49945185 0.1156276 ] omega 0.6153329720201501
transition kernel size 4*4
eigen_vs [1.         0.74759169 0.31935837 0.24860191] omega 0.7475916896131576
transition kernel size 4*4
eigen_vs [1.         0.74121644 0.49945706 0.24123547] omega 0.7412164385072058
transition kernel size 4*4
eigen_vs [1.         0.70152814 0.49082686 0.20324375] omega 0.7015281428333014
transition kernel size 4*4
eigen_vs [1.         0.68825434 0.49899885 0.18850062] omega 0.6882543387791663
transition kernel size 4*4
eigen_vs [1.         0.71810049 0.45141415 0.22329117] omega 0.7181004895907243
transition kernel size 4*4
eigen_vs [1.         0.68960326 0.49953063 0.18971644] omega 0.6896032582481543
transition kernel size 4*4
eigen_vs [